In [1]:
# some imports
import math
import numpy as np
from math import atan2
from collections import namedtuple
from circle_fit import hyper_fit

In [2]:
# We need to define our circle

class Circle():
    def __init__(self, center, radius):
        self.center = center
        self.radius = radius

    def create_array(self, start_angle, end_angle, sample_count=50):
        points = np.zeros((sample_count, 2))
        angles = np.linspace(start_angle, end_angle, sample_count)
        points[:, 0] = self.center.x + np.sin(angles) * self.radius
        points[:, 1] = self.center.y + np.cos(angles) * self.radius
        return points

    def get_angle(self, point):
        return atan2(point.x - self.center.x, point.y - self.center.y)

    def get_closest_point(self, point):
        x = point.x - self.center.x
        y = point.y - self.center.y
        distance = (x**2 + y**2) ** 0.5
        return Point(
            self.center.x + x * self.radius / distance,
            self.center.y + y * self.radius / distance
        )

    @staticmethod
    def fit(points):
        center_x, center_y, radius, _ = hyper_fit(points)
        return Circle(Point(center_x, center_y), radius)

In [3]:
def get_scan_as_cartesian(laser_scan):
    ranges = np.array(laser_scan.ranges)

    angles = np.linspace(
        laser_scan.angle_min,
        laser_scan.angle_max,
        ranges.shape[0])

    laser_range = laser_scan.angle_max - laser_scan.angle_min
    usable_range = math.radians(parameters.usable_laser_range) # TODO parameters
    if usable_range < laser_range:
        skip_left = int((-laser_scan.angle_min - usable_range / 2) / laser_range * ranges.shape[0])
        skip_right = int((laser_scan.angle_max - usable_range / 2) / laser_range * ranges.shape[0])
        angles = angles[skip_left:-1 - skip_right]
        ranges = ranges[skip_left:-1 - skip_right]

    inf_mask = np.isinf(ranges)
    if inf_mask.any():
        ranges = ranges[~inf_mask]
        angles = angles[~inf_mask]

    points = np.zeros((ranges.shape[0], 2))
    points[:, 0] = -np.sin(angles) * ranges
    points[:, 1] = np.cos(angles) * ranges

    return points

In [4]:
def find_left_right_border(points, margin_relative=0.1):
    margin = int(points.shape[0] * margin_relative)

    relative = points[margin + 1:-margin, :] - points[margin:-margin - 1, :]
    distances = np.linalg.norm(relative, axis=1)

    return margin + np.argmax(distances) + 1

In [5]:
def handle_scan(laser_scan, delta_time):
    points = get_scan_as_cartesian(laser_scan)

    if points.shape[0] == 0:
        print("Skipping current laser scan message since it contains no finite values.")
        return

    split = find_left_right_border(points)

    right_wall = points[:split:4, :]
    left_wall = points[split::4, :]

    left_circle = Circle.fit(left_wall)
    right_circle = Circle.fit(right_wall)

    barrier_start = int(points.shape[0] * (0.5 - parameters.barrier_size_realtive)) # TODO parameters
    barrier_end = int(points.shape[0] * (0.5 + parameters.barrier_size_realtive))  # TODO parameters
    barrier = np.max(points[barrier_start: barrier_end, 1])

    follow_walls(left_circle, right_circle, barrier, delta_time)

    show_circle_in_rviz(left_circle, left_wall, 0)
    show_circle_in_rviz(right_circle, right_wall, 1)

In [6]:
# lastly import sample scan data and start the wallfollowing
# TODO import sample scan data
# what does delta_time do?
delta_time = 0
handle_scan(scan_message, delta_time)

NameError: name 'scan_message' is not defined